In [1]:
# Install required packages
!python -m pip install langchain openai mlflow yfinance duckduckgo-search langchain_community ddgs


In [ ]:
import yfinance as yf
from duckduckgo_search import DDGS
from langchain.chat_models import AzureChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import ChatPromptTemplate
from pydantic import BaseModel
import mlflow
import os
import json
import certifi
from ddgs import DDGS
from dotenv import load_dotenv
load_dotenv()
from openai import AzureOpenAI
from langchain_openai import AzureChatOpenAI



python-dotenv could not parse statement starting at line 17


True

In [3]:
import requests
from typing import List, Dict
 
def search_yahoo_finance(query: str, limit: int = 10) -> List[Dict]:
    """
    Search Yahoo Finance for stock symbols by company name.
    Works for multiple exchanges (NSE, BSE, NASDAQ, NYSE, LSE, etc.)
    """
    url = "https://query2.finance.yahoo.com/v1/finance/search"
    params = {"q": query, "quotesCount": limit, "newsCount": 0}
    headers = {"User-Agent": "Mozilla/5.0"}  # pretend like a browser
 
    resp = requests.get(url, params=params, headers=headers, timeout=10)
    resp.raise_for_status()
    data = resp.json()
 
    results = []
    for q in data.get("quotes", [])[:limit]:
        results.append({
            "symbol": q.get("symbol"),
            "name": q.get("shortname") or q.get("longname"),
            "exchange": q.get("exchange"),
            "exchange_display": q.get("exchDisp"),
            "type": q.get("quoteType"),
        })
    return results


In [4]:
STOCK_CODE=search_yahoo_finance("Apple", limit=1)
company_name = STOCK_CODE[0]['name']
stock_code = STOCK_CODE[0]['symbol']
print(STOCK_CODE)

[{'symbol': 'AAPL', 'name': 'Apple Inc.', 'exchange': 'NMS', 'exchange_display': 'NASDAQ', 'type': 'EQUITY'}]


In [17]:
# Step 3: Fetch company news using DuckDuckGo
def fetch_news(company_name):
    results = DDGS().news(company_name, max_results=2)
    return [r["title"] + ": " + r["body"] for r in results]

news_summaries = fetch_news(company_name)
news_text = "\n".join(news_summaries)
print(news_text)

Apple Unveils iPhone 17 as Cramer Labels It His ‘Quarterback Stock’: Apple Inc . (NASDAQ:AAPL) is one of the 12 Jim Cramer Stock Picks this Week. The company unveiled its...
Jim Cramer Gushes About Apple Inc. (AAPL)’s iPhone Air: We recently published 8 Stocks on Jim Cramer’s Radar. Apple Inc . (NASDAQ:AAPL) is one of the stocks ...


In [55]:
# Step 4: Define output schema
class SentimentOutput(BaseModel):
    company_name: str
    stock_code: str
    newsdesc: str
    sentiment: str
    people_names: list[str]
    places_names: list[str]
    other_companies_referred: list[str]
    related_industries: list[str]
    market_implications: str
    confidence_score: float

parser = PydanticOutputParser(pydantic_object=SentimentOutput)


In [8]:
mlflow.set_tracking_uri("http://20.75.92.162:5000/")
print(mlflow.__version__)
mlflow.set_experiment('nikhil_Real-Time Market Sentiment Analyzer Using LangChain Chains')

3.3.2


<Experiment: artifact_location='mlflow-artifacts:/118450250910526494', creation_time=1758007942357, experiment_id='118450250910526494', last_update_time=1758007942357, lifecycle_stage='active', name='nikhil_Real-Time Market Sentiment Analyzer Using LangChain Chains', tags={}>

In [ ]:
from langchain.output_parsers import OutputFixingParser

# Prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a financial sentiment analysis assistant. Respond ONLY with valid dictionary"),
    ("human", " provide a structured sentiment profile:\nNews: {news}\nStock Code: {stock_code}")
])


# Model
model = AzureChatOpenAI(model=model_name)

parser = OutputFixingParser.from_llm(parser=SentimentOutput, llm=model)

# Chain
chain = prompt | model | parser




mlflow.openai.autolog()
with mlflow.start_run(run_name="nikhilpatil-Real-Time Market Sentiment Analyzer Using LangChain Chains"):
    mlflow.log_param("company_name", company_name)
    mlflow.log_param("stock_code", stock_code)
    mlflow.log_text(news_text, "news.txt")

    result = chain.invoke({
    "news": news_text,
    "stock_code": stock_code
})
    mlflow.log_dict(result, "sentiment.json")

    print(result)

{
  "stock": "AAPL",
  "news": "Apple Unveils iPhone 17 as Cramer Labels It His ‘Quarterback Stock’",
  "analyst": "Jim Cramer",
  "sentiment": {
    "overall": "positive",
    "specifics": {
      "product_launch": "positive",
      "analyst_opinion": "favorable",
      "market_reaction": "anticipated to be positive"
    }
  }
}


Trace(trace_id=tr-8b597d359d7742c3cb4d9585c8e245bd)